In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os, glob
import h5py
import fsspec
import datetime
from datetime import datetime
from pyproj import Proj, CRS
import matplotlib.pyplot as plt
import shapely.wkt as wkt
from shapely import geometry, ops
import pandas as pd
import xarray as xr
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_origin
from rasterio import merge
import folium
from folium import plugins
import rioxarray
import matplotlib as mpl
import sys
sys.path.append('../../')
from src.cslc_utils import get_s3path, read_cslc, cslc_info, rasterWrite, custom_merge, colorize, getbasemaps, moving_window_mean

import warnings
warnings.filterwarnings('ignore')

In [3]:
data_dir = 's3://opera-pst-rs-pop1/products/CSLC_S1'
burst_id = ['T144-308014-IW2', 
            'T144-308015-IW2']

# get list of dates
b1paths = (open(f"{burst_id[0]}.txt","r").read().split("\n"))
b2paths = (open(f"{burst_id[1]}.txt","r").read().split("\n"))
dates = [path[40:48] for path in b1paths]
nd = len(dates)

# send to dataframe
file_df = pd.DataFrame({'Date':dates,f'{burst_id[0]}':b1paths,f'{burst_id[1]}':b2paths})

# Check if dir to store tiffs exists and make if not
if os.path.exists("TIFFs") == False:
    os.mkdir('TIFFs')

In [4]:
# get intersect of two bursts and keep polygon variable
cslc_polys = []
for i, id in enumerate(burst_id):
    path_h5 = file_df[f'{id}'][0][:-1]
    cslc = read_cslc(f'{data_dir}/{path_h5}/{path_h5}.h5')

    xcoor, ycoor, dx, dy, epsg, bounding_polygon, orbit_direction = cslc_info(f'{data_dir}/{path_h5}/{path_h5}.h5')
    cslc_poly = wkt.loads(bounding_polygon)
    cslc_polys.append(cslc_poly)
    bbox = [cslc_poly.bounds[0], cslc_poly.bounds[2], cslc_poly.bounds[1], cslc_poly.bounds[3]]

intersect = cslc_polys[0].intersection(cslc_polys[1])
merged_line = ops.linemerge(intersect)
buffer_poly = merged_line.buffer(-.02,single_sided=True)
overlap_poly = geometry.Polygon(buffer_poly.exterior.coords[:])
overlap_bbox = [overlap_poly.bounds[0], overlap_poly.bounds[2], overlap_poly.bounds[1], overlap_poly.bounds[3]]

Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T144-308014-IW2_VV_20151127T135951Z_v0.1_20230708T230140Z/OPERA_L2_CSLC-S1A_IW_T144-308014-IW2_VV_20151127T135951Z_v0.1_20230708T230140Z.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T144-308015-IW2_VV_20151127T135954Z_v0.1_20230708T230140Z/OPERA_L2_CSLC-S1A_IW_T144-308015-IW2_VV_20151127T135954Z_v0.1_20230708T230140Z.h5


In [5]:
def crop_cslc(path,cslc1,cslc2):
    ifg = cslc2 * np.conj(cslc1)
    phase = np.angle(ifg)

    ifg_cpx = np.exp(1j * np.nan_to_num(phase))
    coh = np.clip(np.abs(moving_window_mean(ifg_cpx, 10)), 0, 1)

    # Mask nan values and zero values based on merged ifg file
    nan_mask = np.isnan(ifg)
    zero_mask = ifg == 0
    coh[nan_mask] = np.nan
    coh[zero_mask] = 0

    xcoor, ycoor, dx, dy, epsg, bounding_polygon, orbit_direction = cslc_info(f'{data_dir}/{path}/{path}.h5')

    data = xr.DataArray(data=coh,dims=['y','x'],coords=dict(y=("y", ycoor),x=("x", xcoor)))
    data = data.rio.write_crs(f'epsg:{str(epsg)}',inplace=True)
    data = data.rio.reproject("EPSG:4326")

    mask = data.rio.clip([overlap_poly])
    return mask

In [6]:
# sequential ifgs and coherence
imagenames = []

b1path1_h5 = file_df[f'{burst_id[0]}'][0][:-1]
b1cslc1 = read_cslc(f'{data_dir}/{b1path1_h5}/{b1path1_h5}.h5')

b2path1_h5 = file_df[f'{burst_id[1]}'][0][:-1]
b2cslc1 = read_cslc(f'{data_dir}/{b2path1_h5}/{b2path1_h5}.h5')

for i in range(nd-1):
    b1path2_h5 = file_df[f'{burst_id[0]}'][i+1][:-1]
    b1cslc2 = read_cslc(f'{data_dir}/{b1path2_h5}/{b1path2_h5}.h5')
    b1mask = crop_cslc(b1path2_h5,b1cslc1,b1cslc2)

    b2path2_h5 = file_df[f'{burst_id[1]}'][i+1][:-1]
    b2cslc2 = read_cslc(f'{data_dir}/{b2path2_h5}/{b2path2_h5}.h5')
    b2mask = crop_cslc(b2path2_h5,b2cslc1,b2cslc2)

    #matplotlib plots and save to gif
    filename = f'COH_{dates[i+1]}-{dates[i]}_{orbit_direction}.tif'
    # plotting information
    fig, axs = plt.subplots(2,1,figsize=(20,9))#,tight_layout=True)
    fig.tight_layout(rect=[0,0,.9,.89]) 
    plt.rcParams['font.size'] = 14

    ax1 = axs[0]
    im1 = ax1.imshow(b1mask, cmap='gray',interpolation='nearest', origin='upper',extent=overlap_bbox, vmin=0., vmax=1.)
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.set_title(burst_id[0])

    ax2 = axs[1]
    im2 = ax2.imshow(b2mask, cmap='gray',interpolation='nearest', origin='upper',extent=overlap_bbox, vmin=0., vmax=1.)
    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Latitude')
    ax2.set_title(burst_id[1])

    cbar_ax = fig.add_axes([.9, 0.18, 0.015, 0.6]) # left, bot, wid, hgt
    cbar = plt.colorbar(im2,cax=cbar_ax,spacing='uniform',label='Coherence')


    plt.suptitle(filename[:-4],fontsize=18,y=.92)
    plt.savefig('TIFFs/'+filename,dpi=300,bbox_inches='tight')

    imagenames.append(filename)

    b1cslc1 = b1cslc2
    b2cslc1 = b2cslc2

Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T144-308014-IW2_VV_20151127T135951Z_v0.1_20230708T230140Z/OPERA_L2_CSLC-S1A_IW_T144-308014-IW2_VV_20151127T135951Z_v0.1_20230708T230140Z.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T144-308015-IW2_VV_20151127T135954Z_v0.1_20230708T230140Z/OPERA_L2_CSLC-S1A_IW_T144-308015-IW2_VV_20151127T135954Z_v0.1_20230708T230140Z.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T144-308014-IW2_VV_20151221T135950Z_v0.1_20230708T230145Z/OPERA_L2_CSLC-S1A_IW_T144-308014-IW2_VV_20151221T135950Z_v0.1_20230708T230145Z.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T144-308015-IW2_VV_20151221T135953Z_v0.1_20230708T230145Z/OPERA_L2_CSLC-S1A_IW_T144-308015-IW2_VV_20151221T135953Z_v0.1_20230708T230145Z.h5
Streaming: s3://opera-pst-rs-pop1/products/CSLC_S1/OPERA_L2_CSLC-S1A_IW_T144-308014-IW2_VV_20160114T135949Z_v0.1_20230708T230101Z/OPERA_L2_CSLC-S1A_IW_T

In [7]:
imagenames = [os.path.basename(x) for x in glob.glob('TIFFs/COH*')]

In [8]:
imagenames

['COH_20151221-20151127_Descending.tif',
 'COH_20160114-20151221_Descending.tif',
 'COH_20160207-20160114_Descending.tif',
 'COH_20160302-20160207_Descending.tif',
 'COH_20160326-20160302_Descending.tif',
 'COH_20160419-20160326_Descending.tif',
 'COH_20160513-20160419_Descending.tif',
 'COH_20160606-20160513_Descending.tif',
 'COH_20160630-20160606_Descending.tif',
 'COH_20160724-20160630_Descending.tif',
 'COH_20160817-20160724_Descending.tif',
 'COH_20160910-20160817_Descending.tif',
 'COH_20161004-20160910_Descending.tif',
 'COH_20161028-20161004_Descending.tif',
 'COH_20161121-20161028_Descending.tif',
 'COH_20161215-20161121_Descending.tif',
 'COH_20170108-20161215_Descending.tif',
 'COH_20170114-20170108_Descending.tif',
 'COH_20170120-20170114_Descending.tif',
 'COH_20170126-20170120_Descending.tif',
 'COH_20170201-20170126_Descending.tif',
 'COH_20170225-20170201_Descending.tif',
 'COH_20170309-20170225_Descending.tif',
 'COH_20170321-20170309_Descending.tif',
 'COH_20170402-2

In [9]:
path = "TIFFs/"
images = " ".join([path + filename for filename in imagenames])
os.system('convert -alpha deactivate -verbose -delay 50 -loop 0 -density 300 {} {}animatedall.gif'.format(images, path))

TIFFs/COH_20151221-20151127_Descending.tif TIFF 5836x2449 5836x2449+0+0 8-bit TrueColor sRGB 54.5212MiB 0.330u 0:00.319
TIFFs/COH_20160114-20151221_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.230u 0:00.120
TIFFs/COH_20160207-20160114_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.250u 0:00.118
TIFFs/COH_20160302-20160207_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.220u 0:00.114
TIFFs/COH_20160326-20160302_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.240u 0:00.167
TIFFs/COH_20160419-20160326_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.270u 0:00.114
TIFFs/COH_20160513-20160419_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.200u 0:00.113
TIFFs/COH_20160606-20160513_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.230u 0:00.531
TIFFs/COH_20160630-20160606_Descending.t

0

In [11]:
os.system('convert -alpha deactivate -verbose -delay 50 -loop 0 -density 300 {} {}animatedall.mp4'.format(images, path))

TIFFs/COH_20151221-20151127_Descending.tif TIFF 5836x2449 5836x2449+0+0 8-bit TrueColor sRGB 54.5212MiB 0.280u 0:00.101
TIFFs/COH_20160114-20151221_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.200u 0:00.077
TIFFs/COH_20160207-20160114_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.250u 0:00.079
TIFFs/COH_20160302-20160207_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.160u 0:00.075
TIFFs/COH_20160326-20160302_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.210u 0:00.078
TIFFs/COH_20160419-20160326_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.150u 0:00.075
TIFFs/COH_20160513-20160419_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.210u 0:00.078
TIFFs/COH_20160606-20160513_Descending.tif TIFF 5867x2468 5867x2468+0+0 8-bit TrueColor sRGB 55.2361MiB 0.150u 0:00.075
TIFFs/COH_20160630-20160606_Descending.t

256